## Moneyline Data Pre-Processing
#### CSCI 4502 Final Project
#### Ethan Meyer and Ishika Patel

#### Pre-Processing: Creating columns that will match with Keys of Kaggle Data

In [348]:
import pandas as pd
import numpy as np
import os
import glob
import sys
from Levenshtein import distance as lev_distance
from datetime import datetime
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [415]:
# Function to return TeamID value from Kaggle data based on string similarity with moneyline data
def find_TeamID(team_name):
    min_dist = 1e10
    for idx, possible_name in enumerate(team_name_spelling_df['TeamNameSpelling']):
        if lev_distance(team_name, possible_name) < min_dist:
            min_dist = lev_distance(team_name, possible_name)
            min_idx = idx
    return team_name_spelling_df['TeamID'][min_idx]

def find_DayNum(dirty_str_date, season_end_year, _):
    
    # Determine DayZero of Kaggle Season data frame based on moneyline data
    season_idx = seasons_df[seasons_df["Season"] == season_end_year]["DayZero"].index[0]
    str_day_zero = seasons_df["DayZero"][season_idx]
    # Convert to datetime format
    formatted_day_zero = datetime.strptime(str_day_zero,"%Y-%m-%d %H:%M:%S")
    
    # Convert moneyline data to datetime format
    if len(dirty_str_date) == 3:
        dirty_str_date = "0" + dirty_str_date
        
    if int(dirty_str_date[:2]) >= 10:
        year = str(season_end_year - 1)
    else:
        year = str(season_end_year)

        
    clean_str_date = year + "/" + dirty_str_date[:2] + "/" + dirty_str_date[2:]
    formatted_datetime = datetime.strptime(clean_str_date, '%Y/%m/%d')
    
    # Return number of days between two dates
    return (formatted_datetime.date() - formatted_day_zero.date()).days
    
# Function to integrate moneyline data with Kaggle data based on TeamID
def process_moneyline_data(moneyline_filename, end_year):
    # Reading in file into DataFrame
    moneyline_df = pd.read_excel(moneyline_filename)
    
    # Creating DataFrame
    processed_moneyline_df = pd.DataFrame()
    
    # Creating column for Season 
    processed_moneyline_df["Season"] = [end_year] * len(moneyline_df.index)
    
    # Finding DayNum value to integrate with other data
    processed_moneyline_df['DayNum'] = moneyline_df['Date'].astype(str).apply(find_DayNum, args=(end_year,_))
    
    # Finding TeamID value to integrate with other data
    processed_moneyline_df['TeamID'] = moneyline_df['Team'].apply(find_TeamID)
    
    # Adding Moneyline column
    processed_moneyline_df['Moneyline'] = moneyline_df['ML']
    
    
    return processed_moneyline_df  
    

In [417]:
team_name_spelling_df = pd.read_csv("../data/Kaggle-Data/MDataFiles_Stage1/MTeamSpellings_1.csv")
seasons_df = pd.read_csv("../data//Kaggle-Data/MDataFiles_Stage1/MSeasons.csv")
moneyline_files = glob.glob(os.path.join("../data/NCAA-Moneyline/", "*.xls"))
moneyline_files.sort()
list_of_years = list(range(2008, 2023))


pre_proccessed_df = pd.DataFrame()

for file, year in zip(moneyline_files, list_of_years):
    print("Proccessing",year,"season", "\nFile:",file)
    result_df = process_moneyline_data(file, year)
    pre_proccessed_df = pre_proccessed_df.append(result_df, ignore_index=True)



Proccessing 2008 season 
File: ../data/NCAA-Moneyline/2007-08.xls
Proccessing 2009 season 
File: ../data/NCAA-Moneyline/2008-09.xls
Proccessing 2010 season 
File: ../data/NCAA-Moneyline/2009-10.xls
Proccessing 2011 season 
File: ../data/NCAA-Moneyline/2010-11.xls
Proccessing 2012 season 
File: ../data/NCAA-Moneyline/2011-12.xls
Proccessing 2013 season 
File: ../data/NCAA-Moneyline/2012-13.xls
Proccessing 2014 season 
File: ../data/NCAA-Moneyline/2013-14.xls
Proccessing 2015 season 
File: ../data/NCAA-Moneyline/2014-15.xls
Proccessing 2016 season 
File: ../data/NCAA-Moneyline/2015-16.xls
Proccessing 2017 season 
File: ../data/NCAA-Moneyline/2016-17.xls
Proccessing 2018 season 
File: ../data/NCAA-Moneyline/2017-18.xls
Proccessing 2019 season 
File: ../data/NCAA-Moneyline/2018-19.xls
Proccessing 2020 season 
File: ../data/NCAA-Moneyline/2019-20.xls
Proccessing 2021 season 
File: ../data/NCAA-Moneyline/2020-21.xls
Proccessing 2022 season 
File: ../data/NCAA-Moneyline/2021-22.xls


#### Cleaning: Resolving missing/incorrect data 

In [433]:
pre_proccessed_df.isna().sum()

Season        0
DayNum        0
TeamID        0
Moneyline    76
dtype: int64

In [436]:
sum(pre_proccessed_df['Season'].apply(type) != int)

0

In [439]:
sum(pre_proccessed_df['DayNum'].apply(type) != int)

0

In [440]:
sum(pre_proccessed_df['TeamID'].apply(type) != int)

0

In [441]:
sum(pre_proccessed_df['Moneyline'].apply(type) != int)

2365

In [485]:
# Now we investigate the rows of Moneyline that have a non-integer value
non_int_moneylines = pre_proccessed_df[pre_proccessed_df['Moneyline'].apply(type) != int]['Moneyline']

unique_non_ints = []
for val in non_int_moneylines:
    if val not in unique_non_ints:
        unique_non_ints.append(val)

print("Unique non-integer Moneyline values:",unique_non_ints)

Unique non-integer Moneyline values: ['NL', 61.5, 68.5, 'pk', '-145', '-130', '-149', '-', 128.5, -33.5, nan]


In [486]:
# The negative string values and float values make sense... let's drop rows containing any of the others
cleaned_df = pre_proccessed_df[pre_proccessed_df["Moneyline"].str.contains("NL|pk|-|nan")!=True]


In [487]:
cleaned_df.isna().sum()

Season        0
DayNum        0
TeamID        0
Moneyline    76
dtype: int64

In [484]:
# Finally, drop NAs
cleaned_df = cleaned_df.dropna()